### **Problem Understanding**

Geomagnetic storms are caused by the interaction of solar wind with Earth's magnetic field. The resulting disturbances in the geomagnetic field can wreak havoc on GPS systems, satellite communication, electric power transmission, and more. These disturbances are measured by the Disturbance Storm-Time Index, or [Dst](https://www.ngdc.noaa.gov/stp/GEOMAG/dst.html).

The task is to forecast Dst in real-time to help satellite operators, power grid operators, and users of magnetic navigation systems prepare for magnetic disturbances.

The primary input data  is provided by sensor data from two satellites, NASA's [ACE](https://www.swpc.noaa.gov/products/ace-real-time-solar-wind) and NOAA's DSCOVR. This space weather data includes sensor readings related to both the interplanetary magnetic field and plasma from solar wind.


> **The Interplanetary Magnetic Field (IMF)**
>
The interplanetary magnetic field (IMF) plays a huge rule in how the solar wind interacts with Earth’s magnetosphere. In this article we will learn what the interplanetary magnetic field is and how it affects auroral activity here on Earth.

> **The Sun’s magnetic field**
>
During solar minimum, the magnetic field of the Sun looks similar to Earth’s magnetic field. It looks a bit like an ordinary bar magnet with closed lines close to the equator and open field lines near the poles. Scientist call those areas a dipole. The dipole field of the Sun is about as strong as a magnet on a refrigerator (around 50 gauss). The magnetic field of the Earth is about 100 times weaker.
Around solar maximum, when the Sun reaches her maximum activity, many sunspots are visible on the visible solar disk. These sunspots are filled with magnetism and large magnetic field lines which run material along them. These field lines are often hundreds of times stronger than the surrounding dipole. This causes the magnetic field around the Sun to be a very complex magnetic field with many disturbed field lines.
The magnetic field of our Sun doesn’t stay around the Sun itself. The solar wind carries it through the Solar System until it reaches the heliopause. The heliopause is the place where the solar wind comes to a stop and where it collides with the interstellar medium. Because the Sun turns around her axis (once in about 25 days) the interplanetary magnetic field has a spiral shape which is called the Parker Spiral.

> **Bt value**
>
The Bt value of the interplanetary magnetic field indicates the total strength of the interplanetary magnetic field. It is a combined measure of the magnetic field strength in the north-south, east-west, and towards-Sun vs. away-from-Sun directions. The higher this value, the better it is for enhanced geomagnetic conditions. We speak of a moderately strong total interplanetary magnetic field when the Bt exceeds 10nT. Strong values start at 20nT and we speak of a very strong total interplanetary magnetic field when values exceed 30nT. The units are in nano-Tesla (nT) — named after Nikola Tesla, the famous physicist, engineer and inventor.

> **Bx, By and Bz**
>
The interplanetary magnetic field is a vector quantity with a three axis component, two of which (Bx and By) are orientated parallel to the ecliptic. The Bx and By components are not important for auroral activity and are therefor not featured on our website. The third component, the Bz value is perpendicular to the ecliptic and is created by waves and other disturbances in the solar wind.

![im1](https://www.spaceweatherlive.com/images/help/IMF/BxByBz.gif)

> **Interaction with Earth’s magnetosphere**
>
The north-south direction of the interplanetary magnetic field (Bz) is the most important ingredient for auroral activity. When the north-south direction (Bz) of the the interplanetary magnetic field is orientated southward, it will connect with Earth’s magnetosphere which points northward. Think of the ordinary bar magnets that you have at home. Two opposite poles attract each other! A (strong) southward Bz can create havoc with Earth’s magnetic field, disrupting the magnetosphere and allowing particles to rain down into our atmosphere along Earth’s magnetic field lines. When these particles collide with the oxygen and nitrogen atoms that make up our atmosphere, it causes them to glow and emit light which we see as aurora.
>
For a geomagnetic storm to develop it is vital that the direction of the interplanetary magnetic field (Bz) turns southward. Continues values of -10nT and lower are good indicators that a geomagnetic storm could develop but the lower this value goes the better it is for auroral activity. Only during extreme events with high solar wind speeds it is possible for a geomagnetic storm (Kp5 or higher) to develop with a northward Bz.


![im2](https://www.spaceweatherlive.com/images/help/IMF/magnetosphere.jpg)
 A schematic diagram showing the interaction between the IMF with a southward Bz and Earth’s magnetosphere.

> **Measuring the interplanetary magnetic field**
>
>The real-time solar wind and interplanetary magnetic field data that you can find on this website come from the Deep Space Climate Observatory (DSCOVR) satellite which is stationed in an orbit around the Sun-Earth Lagrange Point 1. This is a point in space which is always located between the Sun and Earth where the gravity of the Sun and Earth have an equal pull on satellites meaning they can remain in a stable orbit around this point. This point is ideal for solar missions like DSCOVR, as this gives DSCOVR the opportunity to measure the parameters of the solar wind and the interplanetary magnetic field before it arrives at Earth. This gives us a 15 to 60 minute warning time (depending on the solar wind speed) as to what kind of solar wind structures are on their way to Earth.
>
>The Deep Space Climate Observatory (DSCOVR) mission is now the primary source for real-time solar wind and interplanetary magnetic field data but there is one more satellite at the Sun-Earth L1 point that measures the incoming solar wind and and that is the Advanced Composition Explorer. This satellite used to be the primary real-time space weather data source up until July 2016 when DSCOVR become fully operational. The Advanced Composition Explorer (ACE) satellite is still collecting data and now operates mostly as a backup to DSCOVR.

*The location of a satellite at the Sun-Earth L1 point.*
![im3](https://www.spaceweatherlive.com/images/help/zonnewind/L1_animation.gif)


In this notebook  we'll cover how to:

- load the data
- create features using the timedelta index
- generate batches of 32-length sequences for training
- train an LSTM model in Keras

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

from scipy import stats
from scipy.stats import norm, skew


from sklearn.model_selection import train_test_split, KFold, GroupKFold, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler

from sklearn.metrics import *

import sys, os
import random 

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
from IPython import display, utils




In [6]:


dst = pd.read_csv("../input/soalr-wind/labels.csv")
dst.timedelta = pd.to_timedelta(dst.timedelta)
# dst.set_index(["period", "timedelta"], inplace=True)


In [7]:
dst.head()

In [8]:
dst.groupby("period").describe()

We have nearly 140,000 observations of hourly dst data, representing over 15 years. There are almost twice as many observations in either the train_b or train_c periods than there are train_a. Also note that most of the values are negative. It also seems train_a represents a more intense period, given that it has a lower mean and higher standard deviation. So we may have outliers.
​
A very strong magnetic field disturbance has a large Dst value, measured in nano-Teslas (nT). Because these disturbances are usually flowing towards the Earth, the values are negative. Sometimes Dst can be highly positive. During calm conditions, Dst values are situated at or just below 0.

### EDA

In [9]:
plt.plot(dst.dst)

We observe that there are many big spikes which are outliers

In [10]:
dst.isnull().sum()

In [11]:
pd.plotting.lag_plot(dst.dst, lag=1)

In [12]:
dst.shape

In [13]:
dst.boxplot(column='dst',by='period')

Removing outliers

In [14]:
q1 = dst['dst'].quantile(0.25)
q3 = dst['dst'].quantile(0.75)
iqr = q3-q1 #Interquartile range
fence_low  = q1-1.5*iqr
fence_high = q3+1.5*iqr
dst_out = dst.loc[(dst['dst'] > fence_low) & (dst['dst'] < fence_high)]


In [15]:
dst_out.head()

In [16]:
dst_out.shape

In [17]:
# df1=dst.dst
# df1.head()

In [18]:
training_size=int(len(dst_out)*0.70)
test_size=len(dst_out)-training_size
# train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]

Splitting into train and test data

In [19]:
def get_train_test_val(data, test_per_period):
    """Splits data across periods into train, test, and validation"""
    # assign the last `test_per_period` rows from each period to test
    test = data.groupby("period").tail(test_per_period)
    train = data[~data.index.isin(test.index)]
    
    return train, test


train_data, test_data = get_train_test_val(dst_out, test_per_period=test_size)
### Modelingm

In [20]:
train_data, test_data=train_data.dst ,test_data.dst

Snipping code for prediction

In [21]:
data_a = dst_out.groupby("period").tail(100)[dst_out['period']=='train_a']
data_b = dst_out.groupby("period").tail(100)[dst_out['period']=='train_b']
data_c = dst_out.groupby("period").tail(100)[dst_out['period']=='train_c']

In [22]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
train_data=scaler.fit_transform(np.array(train_data).reshape(-1,1))
test_data=scaler.transform(np.array(test_data).reshape(-1,1))

In [23]:
train_data.shape

In [24]:
test_data.shape

In [25]:
train_data

In [26]:
test_data

In [27]:
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [28]:
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [29]:
print(X_train.shape), print(y_train.shape)
print(X_test.shape), print(ytest.shape)

In [30]:
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional

In [32]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM,Bidirectional



model = Sequential()
# Adding a Bidirectional LSTM layer
model.add(Bidirectional(LSTM(64,return_sequences=True, dropout=0.5, input_shape=(None,10, 1))))
model.add(Bidirectional(LSTM(32, dropout=0.5)))
model.add(Dense(1))
model.compile(loss='mse', optimizer='rmsprop')
model.build(input_shape=(None,10, 1))

In [33]:
model.summary()

In [34]:
history=model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=30,batch_size=64,verbose=1)

In [35]:
import tensorflow as tf

In [36]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [37]:
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [38]:
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_train,train_predict))

In [39]:
math.sqrt(mean_squared_error(ytest,test_predict))


****Prediction for next 100 days in period train_a

In [40]:
data_a=scaler.transform(np.array(data_a.dst).reshape(-1,1))
data_a.shape

In [41]:
x_input=data_a.reshape(1,-1)
x_input.shape

In [42]:
x_input

In [43]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

In [44]:
temp_input

In [45]:
from numpy import array

lst_output=[]
n_steps=100
i=0
while(i<100):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
#         print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
#         print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
        
        
print(lst_output)

In [46]:
lst_output=scaler.inverse_transform(lst_output)
print(lst_output)

****Prediction for next 100 days in period train_b

In [47]:
data_b=scaler.transform(np.array(data_b.dst).reshape(-1,1))
x_input=data_a.reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()
from numpy import array

lst_output=[]
n_steps=100
i=0
while(i<100):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
#         print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
#         print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
        
        
print(lst_output)

In [48]:
lst_output=scaler.inverse_transform(lst_output)
print(lst_output)

****Prediction for next 100 days in period train_c

In [49]:
data_c=scaler.transform(np.array(data_c.dst).reshape(-1,1))
x_input=data_a.reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()
from numpy import array

lst_output=[]
n_steps=100
i=0
while(i<100):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
#         print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
#         print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
        
        
print(lst_output)

In [50]:
lst_output=scaler.inverse_transform(lst_output)
print(lst_output)

Detailed Eda on the dataset

In [51]:
solar_wind = pd.read_csv("../input/soalr-wind/solar_wind.csv")
# solar_wind.timedelta = pd.to_timedelta(solar_wind.timedelta)
# solar_wind.set_index(["period", "timedelta"], inplace=True)

dst = pd.read_csv("../input/soalr-wind/labels.csv")
# dst.timedelta = pd.to_timedelta(dst.timedelta)
# dst.set_index(["period", "timedelta"], inplace=True)

sunspots = pd.read_csv("../input/soalr-wind/sunspots.csv")
# sunspots.timedelta = pd.to_timedelta(sunspots.timedelta)
# sunspots.set_index(["period", "timedelta"], inplace=True)

satellite_pos = pd.read_csv("../input/soalr-wind/satellite_pos.csv")
# satellite_pos.timedelta = pd.to_timedelta(sunspots.timedelta)
# satellite_pos.set_index(["period", "timedelta"], inplace=True)

In [52]:
dst.head()

In [53]:
dst.groupby("period").describe().T

We have nearly 140,000 observations of hourly dst data, representing over 15 years. There are almost twice as many observations in either the train_b or train_c periods than there are train_a. Also note that most of the values are negative. It also seems train_a represents a more intense period, given that it has a lower mean and higher standard deviation. So we may have outliers.

A very strong magnetic field disturbance has a large Dst value, measured in nano-Teslas (nT). Because these disturbances are usually flowing towards the Earth, the values are negative. Sometimes Dst can be highly positive. During calm conditions, Dst values are situated at or just below 0.

In [54]:
dst.plot(grid=True)

In [55]:
import plotly.express as px
dst_a=dst[dst['period']=="train_a"]
fig=px.line(dst_a,x='timedelta',y='dst', title='Time Series with Rangeslider')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

we observe that generally the value is lying in range (100,0) and (0,-100).

In [56]:
dst_b=dst[dst['period']=="train_b"]
fig=px.line(dst_b,x='timedelta',y='dst', title='Time Series with Rangeslider')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [57]:
dst_c=dst[dst['period']=="train_c"]
fig=px.line(dst_c,x='timedelta',y='dst', title='Time Series with Rangeslider')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [58]:
solar_wind.head()

In [59]:
solar_wind.groupby("period").describe().T.unstack(1)

In [60]:
sunspots.head()

In [61]:
sunspots.groupby("period").describe().T

In [62]:
satellite_pos.head()

In [63]:
satellite_pos.groupby("period").describe().T

We got an insight that values exist across very different scales. For instance, temperature values are quite high - reaching into the hundreds of thousands Kelvin. Meanwhile, IMF readings are fairly small values, and usually negative.

So we have to scale it before training

In [64]:
solar_wind_copy = solar_wind.copy()
dst_copy = dst.copy()
sunspots_copy = sunspots.copy()
satellite_pos_copy = satellite_pos.copy()

In [65]:
solar_wind.timedelta = pd.to_timedelta(solar_wind.timedelta)
solar_wind.set_index(["period", "timedelta"], inplace=True)

dst.timedelta = pd.to_timedelta(dst.timedelta)
dst.set_index(["period", "timedelta"], inplace=True)

sunspots.timedelta = pd.to_timedelta(sunspots.timedelta)
sunspots.set_index(["period", "timedelta"], inplace=True)

satellite_pos.timedelta = pd.to_timedelta(satellite_pos.timedelta)
satellite_pos.set_index(["period","timedelta"], inplace=True)

In [66]:
joined = solar_wind.join(sunspots).join(dst).join(satellite_pos).fillna(method="ffill")

In [67]:
# plt.figure(figsize=(12,10))
# sns.clustermap(joined.corr(), annot=True)

In [68]:
plt.figure(figsize=(20, 20))
ax = sns.heatmap(joined.corr(), annot=True)

The plasma related features like speed and temperature look to be strongly anti-correlated with Dst. The 
IMF feature bt also exhibits strong anti-correlation.
The gsm and gse variables are strongly correlated. This could present multicollinearity issues if both are 
present in our model. We probably want to leave some of those out.
If we had to do multivariate time series analysis, we could have selected following features: speed 
,temperature ,bt , density, gse variables.

In [69]:
dst_copy[dst_copy["period"]=="train_a"]

In [70]:
dst_a=dst_copy[dst_copy['period']=="train_a"]

fig=px.line(dst_a,x='timedelta',y='dst', title='Time Series with Rangeslider')
fig.update_xaxes(rangeslider_visible=True)
fig.show()



In [ ]:
solar_wind_a=solar_wind_copy[solar_wind_copy['period']=="train_a"]
fig=px.line(solar_wind_a,x='timedelta',y='temperature', title='Time Series with Rangeslider')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
dst_a=dst_copy[dst_copy['period']=="train_a"]

fig=px.line(dst_a,x='timedelta',y='dst', title='Time Series with Rangeslider')
fig.update_xaxes(rangeslider_visible=True)
fig.show()



In [ ]:
solar_wind_a=solar_wind_copy[solar_wind_copy['period']=="train_a"]
fig=px.line(solar_wind_a,x='timedelta',y='speed', title='Time Series with Rangeslider')
fig.update_xaxes(rangeslider_visible=True)
fig.show()